# 201805071 BATUHAN EMRE YEŞİLYAYLA
# 201805050 BERK OĞUZ
# BİG DATA FİNAL PROJESİ

In [1]:
!pip install findspark
!pip install psutil

import platform
import socket

hostname = socket.gethostname()
ip_address = socket.gethostbyname(hostname)
computer_name = platform.node()
system_info = platform.uname()
cpu_info = platform.processor()

print("Hostname: ", hostname)
print("IP Address: ", ip_address)
print("Computer Name: ", computer_name)
print("System Info: ")
print(f"  System: {system_info.system}")
print(f"  Node Name: {system_info.node}")
print(f"  Release: {system_info.release}")
print(f"  Version: {system_info.version}")
print(f"  Machine: {system_info.machine}")
print(f"  Processor: {system_info.processor}")
print("CPU Info: ", cpu_info)



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Hostname:  1037136c7610
IP Address:  172.19.0.2
Computer Name:  1037136c7610
System Info: 
  System: Linux
  Node Name: 1037136c7610
  Release: 6.8.0-60-generic
  Version: #63~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Apr 22 19:00:15 UTC 2
  Machine: x86_64
  Processor: 
CPU Info:  


In [2]:
import findspark
findspark.init()
import pyspark 

from pyspark import SparkContext
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [3]:
from pyspark.ml.recommendation import ALS

appName = "Spark Recommend System"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sc=spark.sparkContext
sc

ps: unrecognized option: p
BusyBox v1.37.0 (2025-05-26 20:04:45 UTC) multi-call binary.

Usage: ps [-o COL1,COL2=HEADER] [-T]

Show list of processes

	-o COL1,COL2=HEADER	Select columns for display
	-T			Show threads
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/07 11:58:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=Spark Recommend System>

In [4]:
books = spark.read.csv('data/Books.csv', inferSchema=True, header=True)
users = spark.read.csv('data/Users.csv', inferSchema=True, header=True)
ratings = spark.read.csv('data/Ratings.csv', inferSchema=True, header=True)

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/app/Books.csv.

## 1)PREPROCESSİNG

In [ ]:
books=books.drop("Image-URL-S")
books=books.drop("Image-URL-M")
books=books.drop("Image-URL-L")

books=books.dropna()
books=books.dropDuplicates(subset=["Book-Title"])
books.show(10)

In [ ]:
import pandas as pd

books_pd=books.toPandas()
isbn_mapping = {isbn: index for index, isbn in enumerate(books_pd['ISBN'].unique())}
books_pd['newISBN'] = books_pd['ISBN'].map(isbn_mapping)
print(books_pd.head())


In [ ]:
books_new=spark.createDataFrame(books_pd)

In [ ]:
books_new.show(10)


In [ ]:
books_new.count()

In [ ]:
users=users.dropna()
users=users.dropDuplicates()
users.show(10)
users.count()

In [ ]:
ratings=ratings.dropna()
ratings=ratings.dropDuplicates()
ratings.show(10)
ratings.count()

In [ ]:
spark_df=ratings.join(books_new,"ISBN")
spark_df.show(10)
spark_df.count()

In [ ]:
spark_df=spark_df.dropna()

In [ ]:
spark_df=spark_df.join(users,"User-ID")
spark_df.show(10)
spark_df.count()

In [ ]:
spark_df=spark_df.dropna()

In [ ]:
spark_df.show(10)

In [ ]:
unique_newISBN_count = books_new.select("newISBN").distinct().count()
print("newISBN sütunundaki benzersiz değer sayısı:", unique_newISBN_count)

In [ ]:
spark_df.printSchema()

In [ ]:
from pyspark.sql.functions import col

spark_df = spark_df.filter(col("Book-Rating") != 0)
spark_df = spark_df.filter(col("Book-Author") != "Not Applicable (Na )")
spark_df.show(10)
spark_df.count()

In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import BooleanType

def is_valid_year(year):
    try:
        year = int(year)
        return 1970 <= year <= 2023
    except ValueError:
        return False

is_valid_year_udf = udf(is_valid_year, BooleanType())

filtered_df = spark_df.filter(is_valid_year_udf(col('Year-Of-Publication')))
filtered_df = filtered_df.withColumn('Year-Of-Publication', col('Year-Of-Publication').cast('int'))
filtered_rdd = filtered_df.rdd

mapped_rdd = filtered_rdd.map(lambda row: (row['Year-Of-Publication'], (row['Book-Rating'], 1)))

reduce_output = mapped_rdd.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

average_ratings_rdd = reduce_output.map(lambda x: (x[0], x[1][0] / x[1][1]))

result = average_ratings_rdd.collect()

sorted_result = sorted(result, key=lambda x: x[1], reverse=True)[:10]
top_years_dict = {year: count for year, count in sorted_result}
top_years_dict


In [ ]:
spark_df=filtered_df
spark_df.printSchema()

## 2)Visualition

In [ ]:
df_pandas = pd.DataFrame(list(top_years_dict.items()), columns=['Year', 'Avarage Ratings'])
print(df_pandas)

In [ ]:
import seaborn as sns
sns.barplot(x="Year",y="Avarage Ratings",data=df_pandas)

In [ ]:
rdd = spark_df.rdd
mapped_rdd = rdd.map(lambda row: (row['Year-Of-Publication'], 1))
reduce_output = mapped_rdd.reduceByKey(lambda x, y: x + y)
result = reduce_output.collect()
sorted_result = sorted(result, key=lambda x: x[1], reverse=True)[:10]
top_years_dict_count = {year: count for year, count in sorted_result}
top_years_dict_count

In [ ]:
import pandas as pd
df_pandas = pd.DataFrame(list(top_years_dict_count.items()), columns=['Year', 'Book Count'])
print(df_pandas)

In [ ]:
import seaborn as sns

sns.barplot(x="Year",y="Book Count",data=df_pandas)

In [ ]:
output=spark_df.groupby("Book-Rating").count().toPandas()
sns.barplot(x=output["Book-Rating"],y=output["count"],data=output)

In [ ]:
from pyspark.sql.functions import desc

top_authors_df =books.groupby('Book-Author').count() \
                   .orderBy(desc('count')) \
                   .limit(10)


In [ ]:
top_authors_df.show(10)

In [ ]:
top_authors_pd = top_authors_df.limit(10).toPandas()
print(top_authors_pd)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.barplot(x=top_authors_pd["Book-Author"], y=top_authors_pd["count"], data=top_authors_pd)
plt.xticks(rotation=90)  
plt.xlabel('Yazar')
plt.ylabel('Kitap Sayısı')
plt.title('En Çok Kitabı Basılan Yazar')
plt.show()

In [ ]:
from pyspark.sql.functions import avg
author_book_counts_df = spark_df.groupBy('Book-Author').count()

top_authors_avg_rating_df = spark_df.join(top_authors_df, 'Book-Author') \
                               .groupBy('Book-Author') \
                               .agg(avg('Book-Rating').alias('Average-Rating'))
top_authors_avg_rating_df.show()

In [ ]:
top_authors_avg_rating_pd = top_authors_avg_rating_df.limit(10).toPandas()
top_authors_avg_rating_pd

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x=top_authors_avg_rating_pd["Book-Author"], y=top_authors_avg_rating_pd["Average-Rating"], data=top_authors_avg_rating_pd)
plt.xticks(rotation=90)  
plt.xlabel('Yazar')
plt.ylabel('Kitap Sayısı')
plt.title('En Çok Kitabı Basılan Yazarların Puan Ortalaması')
plt.show()

## 3)ALS Model ile Eğitim

In [ ]:
(train, test) = spark_df.randomSplit([0.7, 0.3])

In [ ]:
train.count()

In [ ]:
test.count()

In [ ]:
spark_df.count()

In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import matplotlib.pyplot as plt
import pandas as pd

ranks = [10, 50, 200]
iterations = [10, 12, 15]
lambdas = [0.01, 0.1]
results = []

best_rmse = float("inf")
best_model = None
best_params = None

for rank in ranks:
    for iteration in iterations:
        for reg_param in lambdas:
            als = ALS(maxIter=iteration, regParam=reg_param, rank=rank, seed=5071,
                      userCol="User-ID", itemCol="newISBN", ratingCol="Book-Rating",
                      coldStartStrategy="drop")

            model = als.fit(train)

            predictions = model.transform(test)
            
            evaluator_rmse = RegressionEvaluator(metricName="rmse", labelCol="Book-Rating", predictionCol="prediction")
            evaluator_mse = RegressionEvaluator(metricName="mse", labelCol="Book-Rating", predictionCol="prediction")
            rmse = evaluator_rmse.evaluate(predictions)
            mse = evaluator_mse.evaluate(predictions)
            
            print(f"Parameters: Rank={rank}, Iteration={iteration}, Lambda={reg_param}")
            print(f"RMSE: {rmse}, MSE: {mse}")
            results.append({"rank": rank, "iteration": iteration, "lambda": reg_param, "rmse": rmse})

            print("---------------------------------------------")
            
            if rmse < best_rmse:
                best_rmse = rmse
                best_model = model
                best_params = {"rank": rank, "iteration": iteration, "lambda": reg_param}

print("Best Parameters:", best_params)
print("Best RMSE:", best_rmse)





In [ ]:
results_df = pd.DataFrame(results)
plt.figure(figsize=(12, 8))
plt.bar(results_df.index, results_df['rmse'], color='blue')
plt.xlabel('Model Index')
plt.ylabel('RMSE')
plt.show()

In [ ]:
model=best_model

In [ ]:
from pyspark.sql.functions import lit


comparison_df = predictions.select("newISBN","Book-Rating", "prediction")

comparison_pd = comparison_df.toPandas()

In [ ]:
comparison_pd.head(20)

## 4)Cosine Similarity

In [ ]:
import numpy as np
from pyspark.sql.functions import col
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

def cosineSimilarity(item_id1, item_id2, a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    cos_similarity = dot_product / (norm_a * norm_b)
    return item_id1, item_id2, cos_similarity

In [ ]:
item_id1 = 53
item_id2 = 53

In [ ]:
item_vector1 = model.itemFactors.filter(col("id") == item_id1).select("features").collect()[0]["features"]
item_vector2 = model.itemFactors.filter(col("id") == item_id2).select("features").collect()[0]["features"]

In [ ]:
cosineSimilarity(item_id1,item_id2,item_vector1,item_vector2)

# 5)Recommend System

In [ ]:
from pyspark.sql.functions import col

def getUsersForBook(bookISBN, numUsers, Users):
    bookUsers = model.recommendForAllItems(numUsers).filter(col("newISBN") == bookISBN)
    userDF = spark.createDataFrame(bookUsers.collect()[0].recommendations)
    
    userDF = userDF.join(Users, userDF["User-ID"] == Users["User-ID"], "inner") \
                           .select(userDF["User-ID"], Users["Location"], Users["Age"], userDF["Rating"])
    
    return userDF



In [ ]:
topUsersForBook = getUsersForBook(146680, 10,users)
topUsersForBook.show()

In [ ]:
train.show(20)